In [1]:
import theano
import numpy as np
import time
import skimage.io

import sys
sys.path.insert(0, '/home/flipvanrijn/Workspace/Dedicon-Thesis/')

%load_ext autoreload
%autoreload 2

Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: Quadro K2200


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision=5, suppress=True)

# Settings
EMBEDDING_SIZE = 512

In [4]:
%%time
from networks.keras.vocab import Vocab

vocab = Vocab('/home/flipvanrijn/Workspace/Dedicon-Thesis/downloads/coco_train_dataset.json')
print 'Vocabulary loaded'

Reduced words from 25128 to 8853 with threshold 5
Vocabulary loaded
CPU times: user 22.7 s, sys: 1 s, total: 23.7 s
Wall time: 23.7 s


In [65]:
from networks.keras.batchgenerator import BatchGenerator

bg = BatchGenerator(vocab)
Xs = bg.get_data(mode='binary', batch_size=5000)
Ys = bg.get_data(mode='index', batch_size=5000)

In [68]:
Xs = Xs.next()
Ys = Ys.next()

In [88]:
print Xs.shape
print Ys[:,:,np.newaxis].shape

(5000, 30, 8856)
(5000, 30, 1)


In [87]:
%%time
from keras.models import Sequential
from keras.layers.core import Dense, Activation, TimeDistributedDense
from keras.layers.recurrent import LSTM, Bidirectional
from keras.regularizers import l2

model = Sequential()
model.add(Bidirectional(LSTM, vocab.token_count, 512, return_sequences=True))
model.add(TimeDistributedDense(512, 1))
model.compile(loss='mse', optimizer='sgd', class_mode='categorical')

CPU times: user 1min 37s, sys: 842 ms, total: 1min 38s
Wall time: 1min 38s


In [90]:
model.fit(Xs, Ys[:, :, np.newaxis], batch_size=128, show_accuracy=True)

Epoch 0
5000/5000 [==============================] - 106s - loss: inf - acc: 1.0000   
Epoch 1
 640/5000 [==>...........................] - ETA: 90s - loss: nan - acc: 1.0000

KeyboardInterrupt: 